# Microbenchmarks

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from benchmarks.runner import BenchmarkRunner

In [ ]:
chis = [8, 16, 32, 64, 128]
reps = 5
svd_results = []
for chi in chis:
    runner = BenchmarkRunner()
    times = []
    mems = []
    for _ in range(reps):
        mat = np.random.randn(chi, chi)
        rec = runner.run_multiple(mat, lambda m: np.linalg.svd(m, full_matrices=False), repetitions=3)
        times.append(rec['run_time'])
        mems.append(rec['run_peak_memory'])
    svd_results.append({
        'chi': chi,
        'mean_time': np.mean(times),
        'std_time': np.std(times),
        'mean_mem': np.mean(mems),
        'std_mem': np.std(mems)
    })
svd_df = pd.DataFrame(svd_results)
svd_df

In [ ]:
plt.figure()
plt.loglog(svd_df['chi'], svd_df['mean_time'], marker='o')
plt.xlabel(r'$\chi$')
plt.ylabel('Runtime (s)')
plt.title('SVD runtime vs bond dimension')
plt.grid(True, which='both')
plt.show()

In [ ]:
frontiers = [10, 50, 200, 1000]
reps = 5
dd_results = []
for r in frontiers:
    runner = BenchmarkRunner()
    times = []
    mems = []
    for _ in range(reps):
        data = np.random.rand(r)
        rec = runner.run_multiple(data, lambda x: np.sort(x), repetitions=3)
        times.append(rec['run_time'])
        mems.append(rec['run_peak_memory'])
    dd_results.append({
        'frontier': r,
        'mean_time': np.mean(times),
        'std_time': np.std(times),
        'mean_mem': np.mean(mems),
        'std_mem': np.std(mems)
    })
dd_df = pd.DataFrame(dd_results)
dd_df

In [ ]:
plt.figure()
plt.plot(dd_df['frontier'], dd_df['mean_time'], marker='o')
plt.xlabel('Frontier size')
plt.ylabel('Runtime (s)')
plt.title('DD frontier extraction time')
plt.grid(True)
plt.show()

In [ ]:
windows = list(range(2, 9))
reps = 5
st_results = []
for w in windows:
    runner = BenchmarkRunner()
    times = []
    mems = []
    for _ in range(reps):
        rec = runner.run_multiple(w, lambda win: sum(range(1 << win)), repetitions=3)
        times.append(rec['run_time'])
        mems.append(rec['run_peak_memory'])
    st_results.append({
        'window': w,
        'mean_time': np.mean(times),
        'std_time': np.std(times),
        'mean_mem': np.mean(mems),
        'std_mem': np.std(mems)
    })
st_df = pd.DataFrame(st_results)
st_df

In [ ]:
st_df

In [ ]:
import pandas as pd
from benchmarks.stats_utils import stats_table

def add_stats(df, quasar_col='QuASAr', baseline_cols=None, test='ttest', correction='bonferroni'):
    """Compute statistics comparing QuASAr with baselines.

    Parameters
    ----------
    df : pandas.DataFrame
        DataFrame with per-circuit results. One column must correspond to QuASAr,
        others to baselines.
    quasar_col : str
        Name of the column containing QuASAr results.
    baseline_cols : list[str] | None
        Columns to treat as baselines. Defaults to all columns except quasar_col.
    test : str
        'ttest' or 'wilcoxon'.
    correction : str
        'bonferroni' or 'fdr_bh'.

    Returns
    -------
    pd.DataFrame
        Table with baseline name, statistic, corrected p-value, and effect size.
    """
    if baseline_cols is None:
        baseline_cols = [c for c in df.columns if c != quasar_col]
    baselines = {c: df[c] for c in baseline_cols}
    return stats_table(df[quasar_col], baselines, test=test, correction=correction)

# Example usage after computing results DataFrame named `results_df`:
# stats_df = add_stats(results_df)
# stats_df


In [ ]:

# Record parameters and results
import json, pathlib
try:
    import ipynbname
    nb_name = ipynbname.path().stem
except Exception:  # pragma: no cover
    nb_name = 'notebook'

# Collect simple parameters from globals
_params = {
    k: v for k, v in globals().items()
    if not k.startswith('_') and isinstance(v, (int, float, str, bool, list, dict, tuple))
}
pathlib.Path('../results').mkdir(exist_ok=True)
with open(f"../results/{nb_name}_params.json", 'w') as f:
    json.dump(_params, f, indent=2)
if 'results' in globals():
    try:
        with open(f"../results/{nb_name}_results.json", 'w') as f:
            json.dump(results, f, indent=2)
    except TypeError:
        pass
print(json.dumps(_params, indent=2))
